In [ ]:
import numpy as np
#LABELING
from sklearn.preprocessing import LabelBinarizer
#TRAIN AND TEST DATASET
from sklearn.model_selection import train_test_split
#EVAL
from sklearn.metrics import classification_report
#DATASET
from sklearn import datasets
#Image processing
import cv2


class NeuralNetwork:
    def __init__(self, layers, alpha=0.1):
        self.W = []
        self.layers = layers
        self.alpha = alpha
        #len layers - 2 to remove in input and output layer
        for i in np.arange(0,len(layers) - 2):
            #get random number
            w = np.random.rand(layers[i] + 1, layers[i+1] + 1)
            # add to W list
            self.W.append(w/np.sqrt(layers[i]))
        w = np.random.rand(layers[-2] + 1, layers[-1])
        self.W.append(w/np.sqrt(layers[-2]))

    #Actiovation funciton for backpropagation
        # sigmoid
    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))
    def sigmoid_deriv(self, x):
        return x*(1-x)
        #Softmax
    def softmax(self, x):
        return np.exp(x)/np.sum(np.exp(x), axis=1, keepdims=True)
    def softmax_deriv(self, x):
        return x*(1-x)
        #tanh
    def tanh(self, x):
        return np.tanh(x)
    def tanh_deriv(self, x):
        return 1.0 - np.tanh(x)**2
    def fit(self, X, y, epochs=1000, displayUpdate=100):
        #Add one
        X = np.c_[X, np.ones((X.shape[0]))]
        for epoch in np.arange(0, epochs):
            for (x, target) in zip(X, y):
                self.fit_partial(x, target)
            if epoch == 0 or (epoch + 1) % displayUpdate == 0:
                loss = self.calculate_loss(X, y)
                print("[INFO] epoch={}, loss={:.7f}".format(epoch + 1, loss))
    def fit_partial(self, x, y):
        #convert to 2D
        A = [np.atleast_2d(x)]
        #FEEDFORWARD
        for layer in np.arange(0, len(self.W)):
            net = A[layer].dot(self.W[layer])
            #if ask to change activation function, change below this
            out = self.sigmoid(net)
            A.append(out)
        error = A[-1] - y
        D = [error*self.sigmoid_deriv(A[-1])]
        #BACKPROPAGATION
        for layer in np.arange(len(A) - 2, 0, -1):
            delta = D[-1].dot(self.W[layer].T)
            delta = delta*self.sigmoid_deriv(A[layer])
            D.append(delta)
        D = D[::-1]
        for layer in np.arange(0, len(self.W)):
            self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])
    def predict(self, X, addBias=True):
        p = np.atleast_2d(X)
        if addBias:
            p = np.c_[p, np.ones((p.shape[0]))]
        for layer in np.arange(0, len(self.W)):
            p = self.sigmoid(np.dot(p, self.W[layer]))
        return p
    def calculate_loss(self, X, targets):
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias=False)
        loss = 0.5 * np.sum((predictions - targets) ** 2)
        return loss
    def save_weights(self, filename):
        np.save(filename, *self.W)
    def load_weights(self, filename):
        self.W = np.load(filename)


"""
#XOR
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [0]])
# Call NN class
nn = NeuralNetwork([2,2,1], alpha=0.5)
#train for XOR
nn.fit(X, y, epochs=20000)
for (x, target) in zip(X, y):
    pred = nn.predict(x)[0][0]
    step = 1 if pred > 0.5 else 0
    print("[INFO] data={}, ground-truth={}, pred={:.4f}, step={}".format(x, target[0], pred, step))
"""

#MNIST dataset
print("INFO loading MNIST (sample dataset...)")
#load datas
digits = datasets.load_digits()
data = digits.data.astype("float")
print("data before:" , data)
#normalize
data = (data - data.min())/(data.max() - data.min())
print("norm data:", data)
print("[INFO] sample: {}, dim: {}".format(data.shape[0], data.shape[1]))
#split train and test
(trainX, testX, trainY, testY) = train_test_split(data, digits.target, test_size=0.25)
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)
#init
nn = NeuralNetwork([trainX.shape[1], 32, 16, 10])

#train

print("[INFO] training network...")
nn.fit(trainX, trainY, epochs=1000)
#eval

print("[INFO] evaluating network...")
print(testX)
print(type(testX))
print(testX.shape)
predictions = nn.predict(testX)
print(predictions.shape)
print(type(predictions))
predictions = predictions.argmax(axis=1)
print("prediction", predictions)
print(classification_report(testY.argmax(axis=1), predictions))

#predict
print("[INFO] predict network...")
"""
# Định dạng danh sách giá trị ảnh 8x8 thành mảng NumPy một chiều
img = [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0]
img = np.array(img)

# In thông tin ảnh và dự đoán
print("[INFO] predict network...")
print(img)
print(img.shape)  # In kích thước của ảnh
# Nếu bạn đã tạo và đào tạo mô hình trước đó (nn) thì bạn có thể tiến hành dự đoán như sau:
# prediction = nn.predict(img.reshape(1, 64))  # Định dạng ảnh thành mảng (1, 64)
# print(prediction.argmax(axis=1))
"""

INFO loading MNIST (sample dataset...)
data before: [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
norm data: [[0.     0.     0.3125 ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.625  0.     0.    ]
 [0.     0.     0.     ... 1.     0.5625 0.    ]
 ...
 [0.     0.     0.0625 ... 0.375  0.     0.    ]
 [0.     0.     0.125  ... 0.75   0.     0.    ]
 [0.     0.     0.625  ... 0.75   0.0625 0.    ]]
[INFO] sample: 1797, dim: 64
[INFO] training network...
[INFO] epoch=1, loss=609.0736953
[INFO] epoch=100, loss=20.2391961
[INFO] epoch=200, loss=3.2125533
[INFO] epoch=300, loss=1.7211749
[INFO] epoch=400, loss=1.4333604
[INFO] epoch=500, loss=1.3067750
[INFO] epoch=600, loss=1.2358314
[INFO] epoch=700, loss=1.1906102
[INFO] epoch=800, loss=1.1593300
[INFO] epoch=900, loss=1.1363993
[INFO] epoch=1000, loss=1.1187800
[INFO] evaluating networ

'\n# Định dạng danh sách giá trị ảnh 8x8 thành mảng NumPy một chiều\nimg = [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0]\nimg = np.array(img)\n\n# In thông tin ảnh và dự đoán\nprint("[INFO] predict network...")\nprint(img)\nprint(img.shape)  # In kích thước của ảnh\n# Nếu bạn đã tạo và đào tạo mô hình trước đó (nn) thì bạn có thể tiến hành dự đoán như sau:\n# prediction = nn.predict(img.reshape(1, 64))  # Định dạng ảnh thành mảng (1, 64)\n# print(prediction.argmax(axis=1))\n'

In [ ]:
print(trainX)

In [ ]:
from PIL import Image
import numpy as np

# Mở ảnh JPEG
image_path = '7777.jpg'  # Thay đổi đường dẫn tới ảnh của bạn
image = Image.open(image_path)

# Chuyển ảnh thành đen trắng (grayscale) và thay đổi kích thước thành 8x8
image = image.convert("L").resize((8, 8))

# Chuyển ảnh thành mảng NumPy
image_data = np.array(image)

# Định dạng lại ảnh thành mảng (1, 64)
image_data = image_data.reshape(1, 64)

# Dự đoán số trên ảnh
prediction = nn.predict(image_data)
predicted_digit = prediction.argmax(axis=1)
print("Predicted digit:", predicted_digit[0])


FileNotFoundError: ignored

In [ ]:
pre = nn.predict(testX).argmax(axis=1)
print(classification_report(testY.argmax(axis=1),pre))



# Open the image using PIL
image = Image.open('00.jpg')
image = image.resize((8,8))
# Convert the image to a NumPy array
pixel_values = np.array(image)

print(pixel_values)
pixel_values = pixel_values.astype("float")

pixel_values = (pixel_values - pixel_values.min())/(pixel_values.max() - pixel_values.min())
print(pixel_values)

print(pixel_values.flatten())
a = nn.predict(pixel_values.flatten()).argmax(axis=1)
print(a)